In [ ]:
--snow pit density
SELECT DISTINCT depth, bottom_depth, value , date 
FROM public.layers WHERE pit_id = 'COGM8C22_20200131' and type = 'density' ORDER BY depth;


In [ ]:
--snow pit temp
SELECT DISTINCT depth, value , date 
FROM public.layers WHERE pit_id = 'COGM8C22_20200131' and type = 'temperature' ORDER BY depth;
 